In [1]:
import copy
import lxml.html
from pathlib import Path
import pickle
import re
import time
from tqdm import tqdm
from stanza.server import CoreNLPClient
from projectmir.xmldocument import XMLDocument

In [2]:
p_path = './data/test_latexml/'
p = Path(p_path)
x = list(p.glob('*.html'))
documents_IDs = [x_.name[:-5] for x_ in x]

docs_list = [[]]*len(documents_IDs)
for i, documents_ID in tqdm(enumerate(documents_IDs)):
    document_path = p_path + documents_ID + '.html'
    start_time = time.time()
    docs_list[i] = XMLDocument(document_path)
    docs_list[i].pattern_based_extract_description()
    if i==0:
        break
save_filename = 'docs_list' + time.strftime('_%d%b%Y_%H%M%S') + '.pkl'
with open(save_filename, 'wb') as f:
    pickle.dump(docs_list, f)

0it [00:00, ?it/s]2020-08-24 07:27:58 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |

2020-08-24 07:27:58 INFO: Use device: cpu
2020-08-24 07:27:58 INFO: Loading: tokenize
2020-08-24 07:27:58 INFO: Done loading processors!
0it [00:39, ?it/s]


loaded data.
processing data...
preprocessed document
extract identifiers...
Number of math components is 50
POS tagging...
Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 600000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-58446ed220a64076.props -preload tokenize,ssplit,pos
extract candidate definition...
Starting server with command: java -Xmx16G -cp /Users/kato/GoogleDrive/project-mir/data/resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-0e9af834d8fd4b2b.props -preload tokenize,ssplit,pos,lemma,ner,depparse
elapsed time: 39.66465 seconds ---
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [ ]:
load_filename = 'docs_list.pkl'
with open(load_filename, 'rb') as f:
    docs_list = pickle.load(f)

In [ ]:
doc0 = docs_list[0]
print(doc0.path)

In [ ]:
# 数式がextract descriptionの対象になると，エラーが出る．
doc0.pattern_based_extract_description()

In [ ]:
description_candidate = 'aaa'
a = description_candidate[-6:-4]

if not a:
    print(type(a))

In [ ]:
description_candidate = 'an inlet temperature MATH0001'
identifier = 'MATH0001'
description_candidate[-8:-4]

In [ ]:
description_candidate[:-8].rstrip()

In [ ]:
description_candidate = description_candidate.replace(')', '\)')

In [ ]:
print(description_candidate)

In [ ]:
pattern_list = [
    re.compile(description_candidate + r' ' + identifier),
    re.compile(identifier + ' is ' + description_candidate),
    re.compile('let ' + identifier + ' be ' + description_candidate),
    re.compile(description_candidate + r' (is|are) denoted by ' + identifier),
    re.compile(identifier + ' denotes ' + description_candidate)
]


In [ ]:
sentence = 'The liquid inlet stream consists of a single component with a mass flow rate MATH0000 and an inlet temperature MATH0001 .'
for pattern_ in pattern_list[:1]:
    if pattern_.findall(sentence):
        print(pattern_.pattern)

In [ ]:
%%time
# sentetnce segmentation using CoreNLP
with CoreNLPClient(
    annotators=['tokenize', 'ssplit'],#, 'pos', 'lemma', 'ner', 'parse', 'depparse', 'coref'],
    timeout=30000,
    memory='16G',
    endpoint='http://localhost:9010') as client:
    ann = client.annotate(doc0.text)
# # get the first sentence
# sentence = ann.sentence[0]
# # print(*[f'id: {token.tokenBeginIndex}\ttext:: {token.word}' for token in sentence.token], sep='\n')
# print(*[f'{token.word}' for token in sentence.token])
for i, sentence in enumerate(ann.sentence):
    # print(f'========== Sentence {i+1} tokens =======')
    # print(*[f'id: {token.tokenBeginIndex}\ttext:: {token.word}' for token in sentence.token], sep='\n')
    print(f'{i}:\t', *[f'{token.word}' for token in sentence.token])


In [ ]:
# https://github.com/stanfordnlp/stanza/issues/288
# get noun phrases with tregex
def noun_phrases(_client, _text, _annotators=None):
    pattern = 'NP'
    matches = _client.tregex(_text, pattern, annotators=_annotators)
    return [sentence[match_id]['spanString'] for sentence in matches['sentences'] for match_id in sentence]
    # print('\n'.join(['\t' + sentence[match_id]['spanString'] for sentence in matches['sentences'] for match_id in sentence]))
with CoreNLPClient(timeout=30000, memory='16G', endpoint='http://localhost:9012') as client:
    sentence_list = doc0.sentences[0]
    sentence = sentence_list[0]
    print(sentence)
    noun_phrase_list = noun_phrases(client, sentence, _annotators='tokenize,ssplit,pos,lemma,parse')



In [ ]:
[i.lstrip() for i in noun_phrase_list]

In [ ]:
doc0.sentences[0][0]

In [ ]:
for identifier, description in zip(doc0.identifiers, doc0.candidate_noun_phrases_list):
    print(identifier)
    for description_ in description:
        for description__ in description_:
            def remove_stopwords(description):
                description_output = description
                # stopwords = []
                # stopwords are tags used in tregex and common stopwords.
                stopwords = ['\n', '(', ')', 'NP', 'DT', 'JJ', 'NN', 'PP', 'IN', 'CC']
                # 参考：https://qiita.com/syunyo/items/2c1ce1d765f46a5c1d72
                
                for stopword in stopwords:
                    description_output = description_output.replace(stopword, '')
                    description_output = description_output.replace('  ', ' ')
                return description_output
            print(remove_stopwords(description__))
            # print(description__.replace('\n', ''))

    print('\n')

In [ ]:
with open('./doc0_text.txt', 'w') as f:
    f.write(doc0.text)
with open('./doc0_body.txt', 'w') as f:
    f.write(doc0.body)

In [ ]:
(doc0.candidate_noun_phrases_list[0][0] + doc0.candidate_noun_phrases_list[0][1])

In [ ]:
with open('./doc0_113130.txt', 'w') as f:
    f.write('\n'.join(doc0.sentences[0]))

In [ ]:
from stanza.server import CoreNLPClient


In [ ]:
def extract_noun_phrases(identifiers, sentences):
    candidate_noun_phrases_list = [[]] * len(identifiers)
    pattern = 'NP'
    with CoreNLPClient(
            annotators=['tokenize', 'ssplit', 'pos', 'lemma', 'ner', 'parse', 'depparse', 'coref'],
            timeout=600000, memory='16G') as client:
        for i, sentence in enumerate(sentences):
            if sentence:
                candidate_noun_phrases_list_ = []
                for text in sentence:
                    matches = client.tregex(text, pattern)
                    matches = matches['sentences'][0]
                    candidates = [matches[match]['match'].rstrip()
                                    for match in matches]
                    candidate_noun_phrases_list_ += candidates

                print('##################')
                print(type(candidate_noun_phrases_list_))
                print(candidate_noun_phrases_list_)
                print('##################')

                candidate_noun_phrases_list[i] = candidate_noun_phrases_list_
    return candidate_noun_phrases_list

In [ ]:
xx = extract_noun_phrases(doc0.identifiers[:1], doc0.sentences[:1])

In [ ]:
print('\n\n'.join(xx[0]))

In [ ]:
doc0.tagged_sentence_list[0]

In [ ]:
doc0.candidate_noun_phrases_list[0][0]
# -> 最初のidentifierに対するnoun candidate
# candidate にidentifierとの距離を埋め込む．-> candidate.extract_noun_phraseとtagged_sentence_listを比較して，長さを計算する．


In [ ]:
# 最初のidentifier[0]が含まれる1つ目の文章[0]の1番目のcandidate[0]
doc0.candidate_noun_phrases_list[0][0][0].replace('NP', '')

In [ ]:
doc0.sentences[0][0]

In [ ]:
import re
import string
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer


def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean


def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs